In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import HTML
HTML("""
<style>
h1,h2,h3 {
	margin: 1em 0 0.5em 0;
	font-weight: 600;
	font-family: 'Titillium Web', sans-serif;
	position: relative;  
	font-size: 36px;
	line-height: 40px;
	padding: 15px 15px 15px 2.5%;
	color: #00018D;
	box-shadow: 
		inset 0 0 0 1px rgba(97,0,45, 1), 
		inset 0 0 5px rgba(53,86,129, 1),
		inset -285px 0 35px #F2D8FF;
	border-radius: 0 10px 0 15px;
	background: #d45d00
    
}
</style>
""")

In [ ]:
from tqdm.notebook import tqdm 
import  matplotlib.pyplot  as plt
import pathlib 
import plotly.express as px
import plotly.offline as py
import plotly.tools as pts
import plotly.graph_objs as go
import seaborn as sns
from wordcloud import WordCloud
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
import missingno as msno
PATH = '/kaggle/input/learnplatform-covid19-impact-on-digital-learning'

In [ ]:
products_info = pd.read_csv(PATH +'/products_info.csv')
districts_info = pd.read_csv(PATH + '/districts_info.csv')

In [ ]:
readme = open(PATH + '/README.md')
readmes = readme.readlines()
display(readmes)

In [ ]:
!ls '/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data'

In [ ]:
## merge the engagement_datas 
p = pathlib.Path(PATH)
datas= list(p.glob('engagement_data/*'))
print('the engagements sum:',len(datas))
en_datas  = []
for en_data in tqdm (datas):
    en_datas.append(pd.read_csv(en_data))
    
engagement_all = pd.concat(en_datas)
display(engagement_all)

                    

# Districts_data

## Exploratory of the data

In [ ]:
print("1、Show The Districts_info info \n")
print('*'*50+'\n')
display(districts_info.info())
print('*'*50+'\n')
print('2、Show The Districts_info isnull \n')
print("*"*50+'*')
display(districts_info.isnull().sum())
print('*'*50+'\n')
print('3、Show The Districts_info duplicates \n')
print('*'*50+'\n')
display(districts_info.duplicated().sum())
print('*'*50+'\n')
print('4、Show The Districts_info basic description \n')
print('*'*50+'\n')
display(districts_info.describe())


## data visualization

In [ ]:
msno.matrix(districts_info,labels=True,fontsize=18)
plt.show()

In [ ]:
msno.bar(districts_info,figsize=(12,8),fontsize=20)
plt.show()

In [ ]:
# the wordcloud
cloud_state = WordCloud(width=2000,height=1200).generate(" ".join(districts_info['state'].astype(str)))
plt.figure(figsize=(40,10))
plt.imshow(cloud_state)
plt.axis('off')

In [ ]:
cloud_locale = WordCloud(width=1500,height=1200).generate(" ".join(districts_info['locale'].astype(str)))
plt.figure(figsize=(40,10))
plt.imshow(cloud_locale)
plt.axis('off')

In [ ]:
plt.figure(figsize=(25,15))
sns.set_theme(style='whitegrid')
ax = sns.countplot(x="state", data=districts_info)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('state',fontsize=25,color='r')
plt.ylabel('count',fontsize=25,color='r')
plt.title('Locale Info Count',fontsize=30,color='g')
plt.show()


In [ ]:
labels = list(districts_info.locale.value_counts().index)
value = districts_info['state'].value_counts()
fig = go.Figure(data=go.Pie(labels=labels,
                             values=value))


fig.update_traces(textposition='inside', hoverinfo='percent+label',textinfo='label+percent')  # 将文本的信息放在里面
fig.update_layout(title='Locale Info',titlefont={'size':25})
fig.show()




In [ ]:
plt.figure(figsize=(25,15))
sns.set_theme(style='whitegrid')
ax = sns.countplot(x="locale", data=districts_info)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('state',fontsize=25,color='g')
plt.ylabel('count',fontsize=25,color='g')
plt.title('State Info Count',fontsize=30,color='g')
plt.show()


In [ ]:
plt.figure(figsize=(25,18))
labels = list(districts_info.state.value_counts().index)
value = districts_info['state'].value_counts()
fig = go.Figure(data=go.Pie(labels=labels,
                             values=value))


fig.update_traces(textposition='inside', hoverinfo='percent+label',textinfo='label+percent')  # 将文本的信息放在里面
fig.update_layout(title='State Info',titlefont={'size':25})
fig.show()


In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=districts_info,x='pct_black/hispanic') 
plt.ylabel('Counts',fontsize=15)
plt.xlabel('pct_black/hispanic',fontsize=15)
plt.show()


In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=districts_info,x='pct_free/reduced') 
plt.ylabel('Counts',fontsize=15)
plt.xlabel('pct_free/reduced',fontsize=15)
plt.show()


In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(data=districts_info,x='county_connections_ratio') 
plt.ylabel('Counts',fontsize=15)
plt.xlabel('county_connections_ratio',fontsize=15)
plt.show()

In [ ]:
## loading the data analysis
from sklearn.preprocessing import LabelEncoder
list_map = []
districts = districts_info.copy()
value1=list(districts.state)
class_le = LabelEncoder()
int_state1 = class_le.fit_transform(value1)
list_map.append(int_state1)
value2=list(districts.locale)
int_state2 = class_le.fit_transform(value2)
list_map.append(int_state2)
value3=list(districts['pct_black/hispanic'])
int_state3 = class_le.fit_transform(value3)
list_map.append(int_state3)
value4=list(districts['pct_free/reduced'])
int_state4 = class_le.fit_transform(value4)
list_map.append(int_state4)
value5=list(districts['county_connections_ratio'])
int_state5 = class_le.fit_transform(value5)
list_map.append(int_state5)
df_map = pd.DataFrame({'state':int_state1,'locale':int_state2,'pct_black/hispanic':int_state3,'pct_free/reduced':int_state4,'county_connections_ratio':int_state5})
plt.figure(figsize = (15,10))
sns.heatmap(df_map.corr())
plt.yticks(rotation = 360,fontsize=15)
plt.xticks(rotation=90,fontsize=15)
plt.title("The districts's data correlation \n")
plt.show()


In [ ]:

import folium
north_america_map = folium.Map(location=[38.9, -77.05], tiles='https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', zoom_start=3,attr='default')
north_america_map


In [ ]:
locations = pd.DataFrame({'State':districts_info['state'].unique()})


from geopy.geocoders import Nominatim

geolocator=Nominatim(user_agent="app")
lat=[]
lon=[]
for location in locations['State']:
    location = geolocator.geocode(location)    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)



locations['lat'] = lat
locations['lon'] = lon




In [ ]:
locations_again = pd.DataFrame(districts_info['state'].value_counts().reset_index())
locations_again.columns=['State','count']
location=locations_again.merge(locations,on='State',how="left").dropna()


In [ ]:
FastMarkerCluster(location[['lat','lon','count']],zoom=20,radius=20).add_to(north_america_map)
north_america_map

# Products_info

## Exploratory the data

In [ ]:
print("1、Show The Products_info info \n")
print('*'*50+'\n')
display(products_info.info())
print('*'*50+'\n')
print('2、Show The Products_info isnull \n')
print("*"*50+'*')
display(products_info.isnull().sum())
print('*'*50+'\n')
print('3、Show The Products_info duplicates \n')
print('*'*50+'\n')
display(products_info.duplicated().sum())
print('*'*50+'\n')
print('4、Show The Products_info basic description \n')
print('*'*50+'\n')
display(products_info.describe())


## data visualization

In [ ]:
print('\n')
plt.figure(figsize=(15,12))
msno.matrix(products_info,labels=True,fontsize=15)
plt.title('The Matrix of the missing value',fontsize=20)
plt.figure(figsize=(15,12))

In [ ]:
print('\n')
plt.figure(figsize=(15,12))
msno.bar(products_info,fontsize=15)
plt.title('The bar of the missing value',fontsize=20)
plt.show()

In [ ]:

plt.figure(figsize=(18,15))
labels = list(products_info['Sector(s)'].value_counts().index)
values = products_info['Sector(s)'].value_counts().values
colors = ['#FF0000', '#E066FF']

trace = [go.Pie(labels = labels, 
                values = values,
                rotation = 30,
                opacity = 1,
                showlegend = True,
                hoverinfo = 'label+percent', 
                textinfo = 'percent',#textinfo = 'value',
                insidetextorientation='radial',
                textfont = dict(size = 30, color = 'white'),
                marker = dict(colors = colors, 
                   line = dict(color = '#000000',width=2)))]
fig = go.Figure(data = trace)
fig.show()

In [ ]:
plt.figure(figsize=(20,15))
labels = list(products_info['Primary Essential Function'].value_counts().index)
values = products_info['Primary Essential Function'].value_counts().values
fig = px.pie(products_info, values=values, names=labels,hole=.3,title='The Primary Essential Function distribution')

fig.update_traces(textposition='inside') # 文字信息在里面
fig.update_layout(uniformtext_minsize=15,  # 文本信息的最小值
                  uniformtext_mode='hide'  # 小于最小值则被隐藏
                 )


In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(y='Primary Essential Function', data=products_info, order=products_info["Primary Essential Function"].value_counts().index[:15])
plt.title("The Primary Essential Function distribution",font="Serif", size=20)
plt.show()


In [ ]:
## loading the data analysis
from sklearn.preprocessing import LabelEncoder
pro_list = []
products = products_info.copy()
value1=list(products['LP ID'])
class_le = LabelEncoder()
int_state1 = class_le.fit_transform(value1)
pro_list.append(int_state1)
value2=list(products.URL)
int_state2 = class_le.fit_transform(value2)
pro_list.append(int_state2)
value3=list(products['Product Name'])
int_state3 = class_le.fit_transform(value3)
pro_list.append(int_state3)
value4=list(products['Provider/Company Name'])
int_state4 = class_le.fit_transform(value4)
pro_list.append(int_state4)
value5=list(products['Sector(s)'])
int_state5 = class_le.fit_transform(value5)
list_map.append(int_state5)
value6=list(products['Primary Essential Function'])
int_state6 = class_le.fit_transform(value6)
pro_list.append(int_state6)
df_map = pd.DataFrame({'state':int_state1,'locale':int_state2,'pct_black/hispanic':int_state3,'pct_free/reduced':int_state4,'county_connections_ratio':int_state5,'Primary Essential Function':int_state6})
plt.figure(figsize = (15,10))
sns.heatmap(df_map.corr())
plt.yticks(rotation = 360,fontsize=15)
plt.xticks(rotation=90,fontsize=15)
plt.title("The products's data correlation \n")
plt.show()

# Engagement_all data

In [ ]:
display(engagement_all)
print("1、Show The Engagement_all info \n")
print('*'*50+'\n')
display(engagement_all.info())
print('*'*50+'\n')
print('2、Show The Engagement_all isnull \n')
print("*"*50+'*')
display(engagement_all.isnull().sum())
print('*'*50+'\n')
print('3、Show The Engagement_all duplicates \n')
print('*'*50+'\n')
display(engagement_all.duplicated().sum())
print('*'*50+'\n')
print('4、Show The Engagement_all basic description \n')
print('*'*50+'\n')
display(engagement_all.describe())


In [ ]:
print("The product_info's lp_id nunique:" ,products_info['LP ID'].nunique())
print("The engagement_all's lp_id nunique:",engagement_all.lp_id.nunique())

In [ ]:
engagement_all['date'] = pd.to_datetime(engagement_all['time']).dt.date
engagement_all['month']= pd.to_datetime(engagement_all['time']).dt.month_name()
engagement_all['weekday']= pd.to_datetime(engagement_all['time']).dt.day_name()

In [ ]:
display(engagement_all)

In [ ]:
plt.figure(figsize=(15,12))
sns.set_style('whitegrid')
sns.stripplot(x="month", y="pct_access", data=engagement_all)
plt.show()

In [ ]:
plt.figure(figsize=(15,12))
sns.stripplot(x="weekday", y="pct_access",
                   data=engagement_all, palette="Set2", size=20, marker="D",
                   edgecolor="gray", alpha=.25)
plt.show()

<h1 style="background-color:#fec5bb;font-size:20px;color:#00033E;font-weight : bold">✌🏾✌🏾Thank you very much for seeing here. I'm a newcomer. If you like it,Please vote it!! I'll KEEP DOING IT!!</h1>
